# Make text files with PyMol commands
Make text files with commands that can be run in PyMol to read in PDB files with reassigned b-factors according to antibody escape.

This is specifically designed for the polyclonal sera and 6M0J, but could work for other selections with modifications.

In [1]:
import os
from IPython.display import display, HTML
import pandas as pd
import yaml

Read in configuration and PSE config:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    
print(f"Reading PSE specs from {config['pse_config_6m0j']}")
with open(config['pse_config_6m0j']) as f:
    pse_config = yaml.safe_load(f)

Reading PSE specs from ../data/pse_config_6m0j.yaml


Make output directory

In [3]:
os.makedirs(config['pse_dir'], exist_ok=True)

## First create the new coloring scheme.
We want to be able to try out a few different color schemes. 

In [4]:
RBD_color = '#DDDDDD' # light gray
core_ep_color = '#E69F00' # orange
ridge_color = '#004488' # dark blue
nub_color = '#66CCEE' # cyan
  
core_ep = [i for i in range(365,373)]+[i for i in range(382,387)] 
ridge = [455,456]+[i for i in range(471,491)]
nub = [i for i in range(443,453)]+[i for i in range(494,502)] 

epitope_sites = core_ep+ridge+nub
other_sites = [i for i in range(331,532) if i not in epitope_sites]

epitope_dict = {}

for site in core_ep:
    epitope_dict[site] = core_ep_color

for site in ridge:
    epitope_dict[site] = ridge_color

for site in nub:
    epitope_dict[site] = nub_color

for site in other_sites:
    epitope_dict[site] = RBD_color

serum_epitopes = pd.DataFrame([(k, v) for k, v in epitope_dict.items()], columns=['site', 'serum_epitopes'])

df = (pd.read_csv(os.path.join('../', config['site_color_schemes']))
                      .merge(serum_epitopes, on='site')
                     )

# df.to_csv(os.path.join('../', config['site_color_schemes']), index=False)

Define some global parameters:

In [5]:
# all maps should also be aligned to 6m0j
ACE2_pdb = '6m0j'

# view1
view1 = """\nset_view (\
     0.042554848,   -0.306899577,    0.950776815,\
    -0.961419106,    0.246264562,    0.122523390,\
    -0.271744192,   -0.919321299,   -0.284589916,\
     0.000373175,   -0.000989944, -207.051269531,\
   -43.128311157,   27.435991287,   25.006759644,\
  -7602.168945312, 8016.250000000,  -20.000000000 )"""

# which set of data to show, and how to color surfaces.
pymol_specs = {
        'metric' : ['max', 'total'],
        'color_min' : 'white',
        'color_max' : 'red',
        'view' : view1,
        }

# Here are the generic commands we want to write:
```
reinitialize
set seq_view, 0

# set working directory
cd Desktop/serum_structures/

load data/6m0j_b-factor-mean-bind.pdb
load data/6m0j_b-factor-mean-expr.pdb

# fetch the open and closed trimer structures 
fetch 6vyb
fetch 6wps

# create ACE2, RBD_bind, RBD_expr
hide all
create ACE2, 6m0j_b-factor-mean-bind and chain A
create RBD_bind, 6m0j_b-factor-mean-bind and chain E; remove RBD_bind and chain A
create RBD_expr, 6m0j_b-factor-mean-expr and chain E; remove RBD_expr and chain A
delete 6m0j_b-factor-mean-bind
delete 6m0j_b-factor-mean-expr

#align structures - first for closed trimer
align 6wps and chain A, RBD_bind;
show_as cartoon, 6wps;
create closed-trimer_6wps, 6wps and chain A+B+E;
show sticks, closed-trimer_6wps and resn NAG+FUC+BMA+MAN;
remove 6wps; color violetpurple, closed-trimer_6wps;

# align structures - now for open trimer
set_name 6vyb, open-trimer_6vyb
align open-trimer_6vyb and chain B, RBD_bind; show_as cartoon, open-trimer_6vyb; color violetpurple, open-trimer_6vyb
remove open-trimer_6vyb and chain B

# create selection with RBD colored according to RBD subdomain 
create RBD, RBD_bind
select RBM, RBD and resi 437-508
select ACE2_contacts, RBD and resi 417+446+449+453+455+456+475+486+487+489+493+496+498+500+501+502+505
color 0xE69F00, RBD
color 0x66CCEE, RBM
color 0x004488, ACE2_contacts
as cartoon, RBD
as cartoon, ACE2
color gray50, ACE2; set cartoon_transparency, 0.2, ACE2

# show RBD_bind and RBD_expr as spectrum b
show surface, RBD_bind; spectrum b, red white, RBD_bind, minimum=-2, maximum=0; show sticks, RBD_bind and resn NAG
show surface, RBD_expr; spectrum b, red white, RBD_expr, minimum=-2, maximum=0; show sticks, RBD_expr and resn NAG

# load total and max escape for each serum

# show each serum as surface, colored by spectrum b

# get the escape sites for all sera (early) and show alpha carbon as sphere

# get the escape sites for all sera (all timepoints) and show alpha carbon as sphere

```

Write function to perform write the same generic commands to text file for every antibody:

In [6]:
def initialize_commands():
    text = """
#commands to load pdbs

reinitialize
set seq_view, 0
set ray_shadows, 0
set spec_reflect, 1
set spec_power, 100000
set sphere_scale, 1

# set working directory
cd Desktop/serum_structures/

load data/6m0j_b-factor-mean-bind.pdb
load data/6m0j_b-factor-mean-expr.pdb

# fetch the open and closed trimer structures
fetch 6vyb
fetch 6wps

# create ACE2, RBD_bind, RBD_expr
hide all
create ACE2, 6m0j_b-factor-mean-bind and chain A
create RBD_bind, 6m0j_b-factor-mean-bind and chain E; remove RBD_bind and chain A
create RBD_expr, 6m0j_b-factor-mean-expr and chain E; remove RBD_expr and chain A
delete 6m0j_b-factor-mean-bind
delete 6m0j_b-factor-mean-expr

#align structures - first for closed trimer
align 6wps and chain A, RBD_bind;
show_as cartoon, 6wps;
create closed-trimer_6wps, 6wps and chain A+B+E;
show sticks, closed-trimer_6wps and resn NAG+FUC+BMA+MAN;
remove 6wps; color violetpurple, closed-trimer_6wps;

# align structures - now for open trimer
set_name 6vyb, open-trimer_6vyb
align open-trimer_6vyb and chain B, RBD_bind; show_as cartoon, open-trimer_6vyb; color violetpurple, open-trimer_6vyb
remove open-trimer_6vyb and chain B

# create selection with RBD colored according to RBD subdomain
create RBD, RBD_bind
# select RBM, RBD and resi 437-508
# select ACE2_contacts, RBD and resi 417+446+449+453+455+456+475+486+487+489+493+496+498+500+501+502+505
# color 0xE69F00, RBD
# color 0x66CCEE, RBM
# color 0x004488, ACE2_contacts

# instead let's try coloring according to arbitrarily-defined epitope sites 
color 0xDDDDDD, RBD
sele core_ep, RBD and resi 365-372+382-386; color 0xE69F00, core_ep
sele ridge, RBD and resi 455+456+471-490; color 0x004488, ridge
sele nub, RBD and resi 443-452+494-501; color 0x66CCEE, nub

as cartoon, RBD
as cartoon, ACE2
color gray50, ACE2; set cartoon_transparency, 0.2, ACE2

create ACE2_ct, ACE2_contacts
as surface, ACE2_ct

# show RBD_bind and RBD_expr as spectrum b
show surface, RBD_bind; spectrum b, red white, RBD_bind, minimum=-2, maximum=0; show sticks, RBD_bind and resn NAG
show surface, RBD_expr; spectrum b, red white, RBD_expr, minimum=-2, maximum=0; show sticks, RBD_expr and resn NAG
""" 
    f.write(text)

Write function to read in each PDB for each selection and max or total escape. 

I am going to write this so the function has to be run each time for `max` and `total`.

Might be able to use formatting like this [example](https://stackoverflow.com/questions/16162383/how-to-easily-write-a-multi-line-file-with-variables-python-2-6/16162599).

In [7]:
def load_escape(sln_list, metric, max_file):
    max_df = pd.read_csv(max_file)
    
    f.write(f'\n\n# load escape PDBs for {metric}\n')
    for sln in sln_list.keys():
        name = sln_list[sln]
        sln_max = max_df.loc[max_df['condition'] == name, 'maximum'].iloc[0]
        
        f.write(f'load data/{sln}_6m0j_{metric}_escape.pdb\n')
        f.write(f'set_name {sln}_6m0j_{metric}_escape, {name}_{metric}\n')
        f.write(f'remove {name}_{metric} and chain A; show surface, {name}_{metric}; show sticks, {name}_{metric} and resn NAG; spectrum b, white red, {name}_{metric}, minimum=0, maximum={sln_max}\n')

Write function to get escape sites for a list of serum selections and an escape threshold (default or sensitive, currently). 

In [8]:
def escape_sites(selection_list, sites_name, escape_threshold, exclude_sites):
    # sites_name is a string (like 'early_escape' that we will call the pymol object)
    
    escape_sites_file = os.path.join('..', config['strong_escape_sites']) 
    escape_sites = list(set(pd.read_csv(escape_sites_file)
                    .query('condition in @selection_list & threshold == @escape_threshold & site not in @exclude_sites')
                    ['site'].astype(str)
                    .tolist()
                   ))
    escape_sites.sort()
    
    sites = '+'.join(escape_sites)
    
    f.write(f'\n# create {sites_name} and show as spheres\n')
    f.write(f'\nsele {sites_name}, RBD and resi {sites}\n')
    f.write(f'show spheres, {sites_name} and name ca\n')

Function to save PNGs for each selection, and rotated 180 degrees. Want something like this:
```
hide all 
show surface, 25_d18_total; show sticks, 25_d18_total and resn NAG
png 25_d18_total_view1_surface.png, ray=1, 600, 600
rotate y, 180
png 25_d18_total_view1_surface_180.png, ray=1, 600, 600
```

In [9]:
def save_images(serum_list, metric):
    for s in serum_list:
        f.write('\n\nhide all')
        f.write(f'\nshow surface, {s}_{metric}; show sticks, {s}_{metric} and resn NAG')
        f.write(f'\npng {s}_{metric}_view1_surface.png, ray=1, 600, 600')
        f.write('\nrotate y, 180')
        f.write(f'\npng {s}_{metric}_view1_surface_180.png, ray=1, 600, 600')
        f.write('\nrotate y, 180')

Function to save ACE2 contacts as ray trace

In [10]:
def trace_ACE2():
    text = """

hide all

as surface, ACE2_ct
set ray_trace_mode, 2
set ray_trace_color, black
set ray_opaque_background, 0

png ACE2.png, ray=1, 600, 600

rotate y, 180
png ACE2_180.png, ray=1, 600, 600

# put things back how they belong
rotate y, 180
set ray_trace_mode, 0
    """
    f.write(text)

Loop through all the PDBs in the config and write output file.

In [11]:
for metric in pymol_specs['metric']:

    outFile = os.path.join(config['pse_dir'], f'pymol_commands_serum_{metric}.txt')
    print(f'Writing pymol commands to \n{outFile}')

    f = open(outFile, "w")
    f.write(f"# PyMol commands for serum selections")
    initialize_commands()

    load_escape(pse_config['all_sera'], metric, pse_config['ylim_file'])

    escape_sites(pse_config['early_sera'], 'early_escape', pse_config['threshold'], pse_config['exclude_sites'])
    escape_sites(pse_config['early_late_sera'], 'all_escape', pse_config['threshold'], pse_config['exclude_sites'])

    f.write(f'\n{view1}\n')
    f.write(f'\nsave serum_escape_{metric}.pse\n')
    f.write(f'\n# png view1.png, ray=1, 600, 600')
    
    save_images(list(pse_config['all_sera'].values()), metric)
    
    trace_ACE2()

    f.close()

Writing pymol commands to 
../results/pymol_commands/pymol_commands_serum_max.txt
Writing pymol commands to 
../results/pymol_commands/pymol_commands_serum_total.txt
